In [ ]:
# !pip install json
# !pip install pandas
# !pip install numpy
# !pip install nltk
# !pip install bs4
# !pip install re
# !pip install contractions

import json
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')
import re
from bs4 import BeautifulSoup
import contractions

# Just for visuals
pd.set_option('display.max_colwidth', None)

In [ ]:
#pip install nlpaug
#pip install transformers

In [ ]:
nltk.download('averaged_perceptron_tagger')

In [ ]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas

In [ ]:
import transformers

In [ ]:
from textblob import TextBlob
nltk.download('punkt')

In [ ]:
#imported data
df0 = pd.read_pickle('class0.pkl')
df1 = pd.read_pickle('class1.pkl')

In [ ]:
#created a list of all the facts
str_lis = df1.facts.values.tolist()

In [ ]:
# concatenated all the facts
long_str = ' '.join(str_lis)

In [ ]:
# extracted the nouns from the facts
blob = TextBlob(long_str)
noun_str = blob.noun_phrases

In [ ]:
long_noun = ' '.join(noun_str)

In [ ]:
# created list of nouns
tokenized = nltk.word_tokenize(long_noun)

In [ ]:
long_tokens = ' '.join(tokenized)

In [ ]:
import re
# removed all the non-alphabetic characters if any
regex = re.compile('[^a-zA-Z]')
#First parameter is the replacement, second parameter is your input string
final_str = regex.sub(' ', long_tokens)

In [ ]:
# since our data is case-sensitive, changed all the nouns 1st character to capital
final_str
result = final_str.title()

In [ ]:
final_tokenized = nltk.word_tokenize(result)

In [ ]:
# augmented the data using contextual word embeddigs
# upto 50% of the non-nouns words have been changed
# used bert-base-cased model, since our data is case sensitive
# stopwords parameter is the list of words which will be skipped without substituting, where we gave list of nouns
# selected top 4 similar words from word embeddings
final_facts = []
for i in range(len(str_lis)):
  TOPK=4 #default=100
  ACT = 'substitute' #"substitute"
  aug_bert = naw.ContextualWordEmbsAug(
      model_path='bert-base-cased', aug_p=0.5,
      stopwords=final_tokenized,
      device='cuda',
      action=ACT, top_k=TOPK)
  print(i)
  #print("Original:")
  #print(test_sentence)
  #print("Augmented Text:")
  final_facts.append(str_lis[i])
  for j in range(4):
    augmented_text = aug_bert.augment(str_lis[i])
    final_facts.append(augmented_text)
    #print(augmented_text)

In [ ]:
df1_few = df1.drop(['facts'], axis=1)

In [ ]:
lis_data = df1_few.values.tolist()

In [ ]:
flis = []
for i in range(len(lis_data)):
  for j in range(5):
    flis.append(lis_data[i])

In [ ]:
fdf = pd.DataFrame(flis, columns =['index', 'ID', 'name', 'href', 'first_party', 'second_party', 'winning_party', 'winner_index'])

In [ ]:
factdf = pd.DataFrame(final_facts, columns =['Facts'])

In [ ]:
fdf['Facts'] = factdf['Facts']

In [ ]:
df0_few1 = df0.drop(['facts'], axis=1)

In [ ]:
df0_few1['Facts'] = df0['facts']

In [ ]:
result1 = df0_few1.append(fdf)

In [ ]:
result1.to_pickle("task1_data.pkl")